In [123]:
import math
import random
import numpy as np
import sys
import copy

In [16]:
file_path_tmp = "../../data/mtsp51.txt"
def load_data(file_path_tmp):
    row_data = []
    with open(file_path_tmp, "r") as f:
        row_data = f.readlines()
    
    row_num = len(row_data) - 1
    
    iso_tuple_list = []
    for i in range(1, len(row_data)):
        item = row_data[i].strip().split(" ")
        iso_tuple_list.append((int(item[0]), int(item[1]), int(item[2])))
    return iso_tuple_list

def cal_dis(t1, t2):
    return math.sqrt(sum([(a - b)**2 for (a,b) in zip(t1[1:],t2[1:])]))

In [47]:
row_data = load_data(file_path_tmp)

N = len(row_data)
M = 30
POPULATION = 100
RUNS = 30

print(row_data[0], row_data[1])
print(cal_dis(row_data[0], row_data[1]))

(1, 37, 52) (2, 49, 49)
12.36931687685298


## Initial

In [90]:
def generate_breaks(n, m, min_nodes, max_nodes = None):
    valid = False
    while valid is False:
        valid = True
#         sorted
        breakpoints = sorted(random.sample(range(n), m - 1))
    
#          first
        if breakpoints[0] < min_nodes or (max_nodes is not None and breakpoints[0] > max_nodes):
            valid = False
            continue
            
#         middle
        for i in range(0, len(breakpoints)-1):
            if breakpoints[i+1] - breakpoints[i] < min_nodes or (max_nodes is not None and breakpoints[i+1] - breakpoints[i] > max_nodes):
                valid = False
                break
        
        if not valid:
            continue
# last
        if n - breakpoints[-1] < min_nodes or (max_nodes is not None and n - breakpoints[-1] > max_nodes):
            valid = False
    return breakpoints

In [104]:
def generate_populations(population_size, n, m, min_nodes, max_nodes = None):
    populations = [random.sample(range(n), n) for i in range(population_size)] 
    breaks = [generate_breaks(n, m, min_nodes, max_nodes) for i in range(population_size)]
    return populations, breaks

def cal_fitness(population, break_points, n):
    fitness_list = []
    paths = [0] + break_points + [n]
    fitness_tmp = 0
    for i in range(len(paths) - 1):
        begin = paths[i]#include
        end = paths[i + 1]#not include
        tmp_index = 0
        for i in range(begin, end):
            fitness_tmp += cal_dis(row_data[population[i]], row_data[tmp_index])
            tmp_index = population[i]
        fitness_tmp += cal_dis(row_data[0], row_data[tmp_index])
        
#         fitness_list.append(fitness_tmp)
    return fitness_tmp

populations, breaks = generate_populations(100, N, M, 1)
print(len(populations), len(breaks))
(cal_fitness(populations[0], breaks[0], N))

100 100


2225.929203132257

In [234]:
def flip_insert(population):
    n = len(population)
    ij = sorted(random.sample(range(n + 1), 2))
    population_tmp = copy.deepcopy(population[ij[0]:ij[1]])
    population_tmp.reverse()
    
    del population[ij[0]:ij[1]]
    
    p = random.randint(0, len(population))
    population[p : p] = population_tmp
    
    
    
def swap_insert(population):
    n = len(population)
    ij = sorted(random.sample(range(n + 1), 2))
    print(ij)
    
    population_tmp = copy.deepcopy(population[ij[0]:ij[1]])
    
    tmp = population_tmp[0]
    population_tmp[0] = population_tmp[len(population_tmp) - 1]
    population_tmp[len(population_tmp) - 1] = tmp
    
    del population[ij[0]:ij[1]]
    
    p = random.randint(0, len(population))
    population[p : p] = population_tmp
    
# TODO add here
def left_lide_insert():
    pass

def right_lide_insert():
    pass
    
def IPGA(populations,breaks, fitness_list, n, m, min_nodes, max_nodes):
    offspring = []
    
    shuffle = random.sample(range(len(populations)), len(populations))
    
    for i in range(0, len(populations), 10):
        index_tmp = shuffle[i:i + 10]
        minus = sys.maxsize
        index_best = -1 
        
        for j in range(10):
            index = index_tmp[j]
            index_best = index if fitness_list[index] < minus else index_best
        
        best_populations = [copy.deepcopy(populations[index_best]) for j in range(10)]
        
#         1, nothing

#         2. FlipInsert
        

In [120]:
iters = 100
def ga_start(max_iters ):
    min_nodes = 1
    max_nodes = 30
    
    populations, breaks = generate_populations(POPULATION, N, M, min_nodes, max_nodes)
    global_best = sys.maxsize
    
    fitness_val = []
    
    for i in range(max_iters):
        fitness_list = [cal_fitness(populations[j], breaks[j], N) for j in range(POPULATION)]
        global_best = min(global_best,min(fitness_list))
    
for i in range(1):
    ga_start(1)


for i in range(0,10,2):
    print(i)

In [235]:
min([2,3,65,1])

1

In [236]:
t = [2,3,65,1]

In [237]:
print(t)

[2, 3, 65, 1]


In [240]:
print(t)
swap_insert(t)
print(t)

[2, 65, 1, 3]
[1, 4]
[3, 1, 65, 2]


In [154]:
ij = sorted(random.sample(range(4 + 1),2))
print(ij[0], ij[1])
t_1 = copy.deepcopy(t[ij[0]:ij[1]])
print(t_1, t)
t_1.reverse()
print(t_1, t)
for i in range(ij[0], ij[1]):
    t[i] = t_1[i - ij[0]]
print(t_1, t)
    

0 2
[3, 2] [3, 2, 65, 1]
[2, 3] [3, 2, 65, 1]
[2, 3] [2, 3, 65, 1]


In [192]:
t = [1, 2, 3, 4]
print(t[1:2])
del t[1:2]
print(t)

[2]
[1, 3, 4]
